In [ ]:
import os
import wandb
import torch
import numpy as np

os.chdir("../")
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5,6"

from config import Config  # noqa: E402


api = wandb.Api()
run_path = "lichangyue/ImageNet-VLM-Eval/rkynyh46"
run = api.run(run_path)
config = run.config
cfg = Config()
cfg.__dict__.update(config)


In [7]:
num_targets = cfg.num_targets
root = "./save/"
f = root + run.file(f"{config['model_name']}/perturbation.pth").name
f = f.replace(".pth", "_evaluation.pth")
results = torch.load(f)

In [8]:
results["train_preds"]

['moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'The image features a close-up of a therm',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'The image features a gold and white clock with a',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'Yorkshire terrier',
 'moving van',
 'moving van',
 'moving van',
 'The image features a colorful and vibrant',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'The image features a large clock with a unique design',
 'The image features a close-up of a pressure',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'The image features a close-up of a clock',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'moving van',
 'Yorkshire terrier',
 'Yorkshire terr

In [ ]:
def calc_asr(preds, targets):
    preds = np.array(preds)
    targets = np.array(targets)
    asr_targets = []
    datasize = len(preds) // num_targets
    for i in range(num_targets):
        left = i * datasize
        right = (i + 1) * datasize
        preds_i = preds[left:right]
        targets_i = targets[left:right]
        asr_targets.append((preds_i == targets_i).mean().item())
    print("ASR for each target:", asr_targets)
    print(f"Average ASR: {np.mean(asr_targets):.4f}")
    return asr_targets

In [25]:
train_asr = calc_asr(results["train_preds"], results["train_targets"])

ASR for each target: [0.86, 0.94]
Average ASR: 0.9000


In [26]:
run.summary.update({"Train_ASR": train_asr})

In [27]:
test_asr = calc_asr(results["test_preds"], results["test_targets"])

ASR for each target: [0.5, 0.7]
Average ASR: 0.6000


In [28]:
run.summary.update({"Test_ASR": test_asr})